In [ ]:
import os              
import re
import pandas as pd

In [ ]:
!pip install transformers
!pip install SentencePiece
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

     |████████████████████████████████| 2.3MB 6.8MB/s 
     |████████████████████████████████| 3.3MB 40.4MB/s 
     |████████████████████████████████| 901kB 50.2MB/s 
     |████████████████████████████████| 1.2MB 6.8MB/s 


In [ ]:
os.getcwd()

'/content'

In [ ]:
path=os.getcwd()+'/drive/MyDrive/t5_dia/';
path

'/content/drive/MyDrive/t5_dia/'

In [ ]:
df=pd.read_excel(path+'new_data_to_text_dib.xlsx')
# df=pd.read_csv(path+'output+ve.csv')

In [ ]:
df

,Input,Output
0,<table><cell>0.463309242323455<col-header>gluc...,The patient is adviced to take diabeties medic...
1,<table><cell>0.463309242323455<col-header>gluc...,The patient is adviced to take diabeties medic...
2,<table><cell>0.467772410721092<col-header>gluc...,The patient is at a risk of having diabetes be...
3,<table><cell>0.467772410721092<col-header>gluc...,The patient has chances of diabetes because th...
4,<table><cell>0.0832939061552806<col-header>bmi...,The patient is adviced to take diabeties medic...
...,...,...
1531,<table><cell>-0.0113546653939187<col-header>79...,The patient is absolutely fine because the ins...
1532,<table><cell>-0.0446877883927246<col-header>ag...,The patient is predicted healthy because she f...
1533,<table><cell>-0.0446877883927246<col-header>ag...,The patient is predicted healthy because the p...
1534,<table><cell>-0.00962395783123568<col-header>2...,The patient is not at a risk of having diabete...


In [ ]:
df=df.sample(frac=1)

In [ ]:
df.reset_index(drop=True)

,Input,Output
0,<table><cell>0.467131684197711<col-header>gluc...,The patient has chances of diabetes because th...
1,<table><cell>0.464981728307273<col-header>gluc...,The patient is at a risk of having diabetes be...
2,<table><cell>-0.00359590279056172<col-header>b...,The patient is predicted healthy because the p...
3,<table><cell>-0.0164548187722279<col-header>in...,The patient is not at a risk of having diabete...
4,<table><cell>-0.0227470505579425<col-header>20...,The patient is not at a risk of having diabete...
...,...,...
1531,<table><cell>-0.00831886752391001<col-header>2...,The patient is not at a risk of having diabete...
1532,<table><cell>0.468387847841206<col-header>gluc...,The patient has chances of diabetes because bl...
1533,<table><cell>-0.0778276208506255<col-header>bm...,The patient is not at a risk of having diabete...
1534,<table><cell>-0.0302667355924925<col-header>79...,The patient is absolutely fine because she has...


In [ ]:
df=df.sample(frac=1)
S=int(df.shape[0]/100*80)
train_df=df.iloc[  :S,:]
train_df=train_df.sample(frac = 1)
test_df=df.iloc[S:,:]
test_df=test_df.sample(frac=1)

In [ ]:
1150-920


230

In [ ]:
int(df.shape[0]/100*80)

1228

In [ ]:
batch_size=16
num_of_batches=len(train_df)/batch_size

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
    print("Running on the GPU")
else:
    dev = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

model = T5ForConditionalGeneration.from_pretrained('t5-base',
                                             return_dict=True)

model.to(dev)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [ ]:
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

In [ ]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [ ]:
type(tokenizer)

transformers.models.t5.tokenization_t5.T5Tokenizer

In [ ]:
train_df['Input'][1]

'<table><cell>0.463309242323455<col-header>glucose > 140.00</col-header></cell><cell>0.0237646354638792<col-header>age > 40.25</col-header></cell></table>'

In [ ]:
# for indx,row in train_df.iterrows():
#   print(indx,row)

In [ ]:
dir()

In [ ]:
#Sets the module in training mode
model.train()
num_of_epochs=2
loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
  print('Running epoch: {}'.format(epoch))
  
  running_loss=0
  out = display(progress(1, num_of_batches+1), display_id=True)
  for i in range(int(num_of_batches)):
    inputbatch=[]
    labelbatch=[]
    new_df=train_df[i*batch_size:i*batch_size+batch_size]
    for indx,row in new_df.iterrows():
      input =  row['Input'] 
      labels = row['Output']
      inputbatch.append(input)
      labelbatch.append(labels)
    
    inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True ,max_length=400,return_tensors='pt')["input_ids"]

    labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True ,max_length=400,return_tensors="pt") ["input_ids"]
    inputbatch=inputbatch.to(dev)
    labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
    optimizer.zero_grad()

    # Forward propogation
    outputs = model(input_ids=inputbatch, labels=labelbatch)
    loss = outputs.loss
    loss_num=loss.item()
    logits = outputs.logits
    running_loss+=loss_num
    if i%10 ==0:      
      loss_per_10_steps.append(loss_num)
    out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
    loss.backward()

    #updating the params
    optimizer.step()
    
  running_loss=running_loss/int(num_of_batches)
  print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))

Running epoch: 1


Epoch: 1 , Running loss: 0.12381452272989248
Running epoch: 2


Epoch: 2 , Running loss: 0.11365697621122788


In [ ]:
torch.save(model.state_dict(),path+'pytorch_model.bin')

In [ ]:
# !wget https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json

In [ ]:
tokenizer1 = T5Tokenizer.from_pretrained('t5-base')
# path=os.getcwd()+"/drive/MyDrive/Colab/T-5/"
model1 =T5ForConditionalGeneration.from_pretrained(path+'pytorch_model.bin',return_dict=True,config=path+'config.json')

def generate(text,model1,tokenizer1):
   model.eval()
   input_ids = tokenizer1.encode(text, 
                               return_tensors="pt")  
   outputs = model1.generate(input_ids,max_length=120)
   return tokenizer1.decode(outputs[0])

In [ ]:
test_df['Input']

56     <table><cell>0.464428850701571<col-header>gluc...
927    <table><cell>-0.0029800324607476<col-header>bp...
969    <table><cell>-0.0164548187722279<col-header>in...
275    <table><cell>0.467326646952658<col-header>gluc...
963    <table><cell>-0.0513887143623219<col-header>ag...
                             ...                        
578    <table><cell>-0.0151822422624577<col-header>32...
804    <table><cell>-0.0754989506736372<col-header>bm...
981    <table><cell>-0.0215750123850443<col-header>32...
173    <table><cell>0.46269732401742<col-header>gluco...
90     <table><cell>0.471521354113385<col-header>gluc...
Name: Input, Length: 308, dtype: object

In [ ]:
x=90
s=generate(test_df['Input'][x],model1,tokenizer1)
print(s)
print(test_df['Output'][x])
print(test_df['Input'][x])

<pad> The patient is adviced to take diabeties medicine because blood tests report high glucose and she falls in the indicative age category</s>
The patient is adviced to take diabeties medicine because the patient has high glucose levels and she falls in the indicative age category
<table><cell>0.471521354113385<col-header>glucose > 140.00</col-header></cell><cell>0.0461009811730105<col-header>age > 40.25</col-header></cell></table>


In [ ]:
Truth=[]
Prediction=[]

In [ ]:
for x in range(test_df.shape[0]):
  print("\r{0}%".format(round((x+1)/test_df.shape[0]*100),2),end='')
  real_Output=test_df.iloc[x,:]['Output']
  predicted_output=generate(test_df.iloc[x,:]['Input'],model1,tokenizer1)[6:-4]
  Truth.append(real_Output)
  Prediction.append(predicted_output)


100%

In [ ]:
Predicted_data=pd.DataFrame({'Predicted':Prediction,'Truth':Truth})

In [ ]:
Predicted_data.to_excel(path+'T5_Predicted_Data.xlsx',index=False)

In [ ]:
BS_data=pd.read_excel(path+'T5_Predicted_Data.xlsx')

In [ ]:
!pip uninstall nltk

In [ ]:
!pip install nltk

     |████████████████████████████████| 1.5MB 7.2MB/s 


In [ ]:
BS_data

,Predicted,Truth
0,The patient is adviced to take diabeties medic...,The patient has chances of diabetes because bl...
1,The patient is absolutely fine because the pat...,The patient is absolutely fine because the blo...
2,The patient is absolutely fine because insulin...,The patient is predicted healthy because she h...
3,The patient is adviced to take diabeties medic...,The patient has chances of diabetes because he...
4,The patient is absolutely fine because she fal...,The patient is predicted healthy because she f...
...,...,...
303,The patient is absolutely fine because her bod...,The patient is absolutely fine because her bod...
304,The patient is absolutely fine because patient...,The patient is predicted healthy because patie...
305,The patient is absolutely fine because the pat...,The patient is not at a risk of having diabete...
306,The patient is adviced to take diabeties medic...,The patient has chances of diabetes because th...


In [ ]:
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
predicted_text=BS_data['Predicted']
real_text=BS_data['Truth']
BleuScore=[]
for x in range(len(real_text)):
  BleuScore.append(sentence_bleu([real_text[x]],predicted_text[x]))

In [ ]:
sum(BleuScore)/len(BleuScore)

0.6697928891830792

In [ ]:
from nltk.translate.meteor_score import meteor_score
import nltk

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
predicted_text=BS_data['Predicted']
real_text=BS_data['Truth']
MeteorScore=[]
for x in range(len(real_text)):
  MeteorScore.append(meteor_score([real_text[x]],predicted_text[x]))

In [ ]:
sum(MeteorScore)/len(MeteorScore)

0.5856064534195612

In [ ]:
!pip install rouge_score

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

In [ ]:
predicted_text=BS_data['Predicted']
real_text=BS_data['Truth']
RougeScore=[]
for x in range(len(real_text)):
  RougeScore.append(scorer.score(real_text[x],predicted_text[x]))

In [ ]:
r_precision=0
r_recall=0
r_fmeasure=0
for x in range(len(RougeScore)):
  r_precision+=RougeScore[x]['rouge1'].precision
  r_fmeasure+=RougeScore[x]['rouge1'].fmeasure
  r_recall+=RougeScore[x]['rouge1'].recall

In [ ]:
print('T5')
print('Bleu Score:   ',sum(BleuScore)/len(BleuScore))
print('Meteor Score: ',sum(MeteorScore)/len(MeteorScore))
print('\n')
print('Rouge Score')
print('Precision:    ',r_precision/len(RougeScore))
print('Recall:       ',r_recall/len(RougeScore))
print('fmeasure:     ',r_fmeasure/len(RougeScore))

T5
Bleu Score:    0.6697928891830792
Meteor Score:  0.5856064534195612


Rouge Score
Precision:     0.6768117788920502
Recall:        0.6370317569927361
fmeasure:      0.6527458124521779
